Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [1]:
import qsharp

Run Q# using the `%%qsharp` magic.

`DumpMachine()` and `Message()` output get formatted as HTML. Return value is shown as cell output.

In [2]:
%%qsharp

operation Main() : Result {
    use q = Qubit();
    X(q);
    Microsoft.Quantum.Diagnostics.DumpMachine();
    let r = M(q);
    Message($"The result of the measurement is {r}");
    Reset(q);
    r
}

Main()

STATE:
|1⟩: 1.0000+0.0000𝑖

The result of the measurement is One

One

`qsharp.eval()` does the same thing as the `%%qsharp` magic.

`DumpMachine()` and `Message()` print to stdout and get displayed in the notebook as plain text

In [ ]:
qsharp.eval("Main()")

Assign a result to a Python variable.

In [ ]:
result = qsharp.eval("1 + 2")

print(f"Result: {result} (type: {type(result).__name__})")

Errors are exceptions. 

Catch and handle compilation errors.

In [ ]:
from qsharp import QSharpError

try:
    qsharp.eval(
        """
operation Foo() : Unit {
    Bar();
    Baz();
}
"""
    )
except QSharpError as ex:
    print(ex)

Catch and handle runtime errors.

In [ ]:
try:
    qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")
except QSharpError as ex:
    print(ex)

A runtime error that's not caught gets reported as a Python exception.

In [ ]:
qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")

In `%%qsharp` cells, exceptions are handled and displayed as error text.

In [ ]:
%%qsharp

operation Bar() : Unit {
    use q = Qubit(); 
    Microsoft.Quantum.Diagnostics.DumpMachine(); 
    X(q);
} 
    
Bar()

Streaming output for long running operations.

In [ ]:
%%qsharp

open Microsoft.Quantum.Diagnostics;

operation Main() : Unit {
    Message("Generating random bit... ");
    for i in 0..400000 {
        use q = Qubit();
        H(q);
        let r = M(q);
        if (i % 100000) == 0 {
            DumpMachine();
            Message($"Result: {r}");
        }
        Reset(q);
    }
}

Main()

Running multiple shots for an expression. Each shot uses an independent instance of the simulator. A list of results (or runtime errors) is returned.

In [6]:
%%qsharp

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    let res = M(q);
    Reset(q);
    return res;
}

In [5]:
results = qsharp.run("RandomBit()", shots=10)

results

[Zero, Zero, Zero, One, One, One, Zero, Zero, One, Zero]

The results can then be processed, e.g. plotted in a histogram using popular Python libraries.

In [ ]:
%pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Sort the results so that the histogram labels appear in the correct order
results.sort()
# Count the number of times each result appears
counts = Counter(results)

(values, counts) = counts.keys(), counts.values()
xlabels = np.arange(len(counts))
plt.bar(xlabels, counts)
plt.xticks(xlabels, values)
plt.show()

In [ ]:
%pip install pandas

In [ ]:
import pandas
from collections import Counter

# Sort the results so that the histogram labels appear in the correct order
results.sort()
pandas.Series(results).value_counts(sort=False).plot(kind='bar')

A compiler error in the entry expression:

In [ ]:
qsharp.run("""RandomBit("a")""", 10)

Some shots throw runtime errors:

In [ ]:
%%qsharp

operation Bad() : Unit {
    use q = Qubit();
    H(q);
    let res = M(q);
    if (res == One) {
        // Do something bad, sometimes
        use q2 = Qubit();
        X(q2);
    }
}

In [ ]:
qsharp.run("Bad()", 10)